# ResonFit: Legacy Resonator_Fitter Example

This notebook demonstrates how to use the original ResonatorFitter class to analyze superconducting resonator S21 data. This approach uses a single comprehensive class rather than the modular approach of the main ResonFit package.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the parent directory to sys.path to import ResonatorFitter
# This is only needed for this example - normally you would install the package
sys.path.append(os.path.abspath(os.path.join('.', '..')))
from resonfit.legacy.resonator_fitter import ResonatorFitter  # Assuming we've moved the original file here

## 1. Loading Data

First, let's load our S21 data from a CSV file or generate sample data.

In [ ]:
# File name and cable delay settings
filename = "RES3_70DBM.CSV"  # Replace with your data file name
use_sample_data = True  # Set to False to use your own data file

if use_sample_data:
    # Generate sample data
    def generate_sample_data(noise_level=0.01):
        fr = 5e9  # 5 GHz resonance frequency
        Ql = 10000  # Loaded quality factor
        Qc = 20000  # Coupling quality factor
        phi = 0.1  # Impedance mismatch angle
        
        # Generate frequencies centered around fr
        freqs = np.linspace(fr - fr/Ql*5, fr + fr/Ql*5, 401)
        
        # Delay and background
        delay = 50e-9  # 50 ns
        a = 0.9
        alpha = 0.2
        
        # Generate ideal S21 data
        Qc_complex = Qc * np.exp(-1j * phi)
        s21_ideal = 1 - (Ql / Qc_complex) / (1 + 2j * Ql * (freqs - fr) / fr)
        
        # Add environmental effects
        s21_with_env = a * np.exp(1j * alpha) * s21_ideal * np.exp(1j * 2 * np.pi * freqs * delay)
        
        # Add noise
        noise = noise_level * (np.random.randn(len(freqs)) + 1j * np.random.randn(len(freqs)))
        s21_noisy = s21_with_env + noise
        
        return freqs, s21_noisy, {"fr": fr, "Ql": Ql, "Qc": Qc, "phi": phi}
    
    # Generate sample data
    freqs, s21, true_params = generate_sample_data(noise_level=0.005)
    print("Using generated sample data")
else:
    # Load S21 data from file
    try:
        # Adjust skip_header as needed for your CSV file
        data = np.genfromtxt(filename, skip_header=3, delimiter=',')
        freqs = data[:, 0]
        s21 = data[:, 1] + 1j * data[:, 2]
        print(f"Successfully loaded data from {filename}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        sys.exit(1)
    except Exception as e:
        print(f"Error reading file '{filename}': {e}")
        sys.exit(1)

## 2. Using ResonatorFitter

Now let's use the ResonatorFitter class to analyze our data.

In [ ]:
# Create ResonatorFitter instance
# weight_bandwidth_scale controls how the weights are applied in fitting
fitter = ResonatorFitter(freqs, s21, weight_bandwidth_scale=1.0)

# Run the full fitting procedure (with plots)
final_dcm_params = fitter.full_fit_procedure(plot_all=True, use_dcm_weights=True)

## 3. Displaying Results

Let's display the final results from our fitting procedure.

In [ ]:
if final_dcm_params:
    print("\n--- Final Fitted Parameters ---")
    all_params = fitter.get_parameters()
    for key, value in all_params.items():
        if isinstance(value, (float, np.floating)):
            print(f"{key}: {value:.4f}")
        else:
            print(f"{key}: {value}")
    
    if use_sample_data and 'true_params' in locals():
        print("\n--- Comparison with True Values ---")
        print(f"Resonance Frequency: {true_params['fr']/1e9:.6f} GHz (True) vs {all_params['fr_dcm_GHz']:.6f} GHz (Fitted)")
        print(f"Internal Quality Factor: {true_params['Qc']/2:.1f} (True, approx) vs {all_params['Qi_dcm']:.1f} (Fitted)")
        print(f"Coupling Quality Factor: {true_params['Qc']:.1f} (True) vs {all_params['Qc_mag_dcm']:.1f} (Fitted)")
        print(f"Impedance Mismatch Angle: {true_params['phi']:.4f} rad (True) vs {all_params['phi_dcm_rad']:.4f} rad (Fitted)")

else:
    print("Fitting procedure failed or did not produce results.")

## 4. Comparison with Modular Approach

The original `ResonatorFitter` class combines all functionality in a single class, which was refactored into the modular ResonFit package. The modular approach offers several advantages:

1. **Extensibility**: Easily add new preprocessing steps or fitting methods
2. **Flexibility**: Mix and match components for different analysis needs
3. **Maintainability**: Smaller, focused classes are easier to test and maintain
4. **Reusability**: Components can be used independently or together

For most new applications, we recommend using the modular approach shown in the `basic_usage.ipynb` example.